In [1]:
import tensorflow as tf
import numpy as np

# TFRecord

## 配置函数

In [2]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
def serialize_example(num1,num2,string,ndarray,image):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    feature = {
      'Int': _int64_feature(num1),
      'Float': _float_feature(num2),
      'String': _bytes_feature(string.encode()),
      'Ndarray': _bytes_feature(ndarray.tobytes()),
      'Image_raw': _bytes_feature(image),
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

## 创建数据

In [4]:
from captcha.image import ImageCaptcha
img = ImageCaptcha(width=28,height=28)
img.write("z","test.png")

In [5]:
from PIL import Image
Image.open("test.png")

In [6]:
npa = np.random.randint(1,10,(2,4))  # 注意dtype
npa

array([[1, 9, 9, 8],
       [5, 2, 5, 8]])

In [7]:
serialized_example = serialize_example(99,12.364,"test",npa,open("test.png","rb").read())
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "Float"
    value {
      float_list {
        value: 12.36400032043457
      }
    }
  }
  feature {
    key: "Image_raw"
    value {
      bytes_list {
        value: "\211PNG\r\n\032\n\000\000\000\rIHDR\000\000\000\034\000\000\000\034\010\002\000\000\000\375oH\303\000\000\006\\IDATx\234mV\337o\024\347\025=\337\217\231ov\3273\273\330^c/;f\211\035(6$)\251T\340\001\032\"RWU\243\244*\364\311\352\023o}\350?\320?\242\317\311C$\336@\252T!\245 \267M\325J\320>PR\300\242\300\202\rk\257\261w\355\365\314\356z\347\347w\3730\216qH\256F\232\031it\276s\356=\347j\330\347\177\370\020\373\352\322\374\002\000)\331\255\033\277m\277z\300\271A\000\203\326 \322\321\370\324\317[/\376V\236\370a\263q\333\3409p\304b{\356g_\224GN\244\232\366@$\000\333q\001(\345\234|\377J\024\372B\232\233\255\'\2461$\244\245u\302\030\010Zp\236;P\335j\336\026\322j\277\372\257i\024H\247BZc\345\363\202\311T\'\200\010C\277\3537\276\272\371;yi~\241\331\270\335\\\271sdz\2560TIu\322\363\232\313\365\277z\

## 写入数据

In [8]:
with tf.io.TFRecordWriter("test.tfrecord") as writer:
    for i in range(1):
        writer.write(serialized_example)

In [9]:
# Create a description of the features.
feature_description = {
    'Int': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'Float': tf.io.FixedLenFeature([], tf.float32, default_value=0.),
    'String': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'Ndarray': tf.io.FixedLenFeature([], tf.string),
    'Image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_function(example_proto):
  # Parse the input `tf.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

## 读取数据

In [10]:
raw_dataset = tf.data.TFRecordDataset("test.tfrecord")
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [11]:
parsed_dataset = raw_dataset.map(_parse_function)

In [12]:
for parsed_record in parsed_dataset.batch(10):
    print(parsed_record["Int"])
    print(parsed_record["Float"])
    print(parsed_record["String"])
    npb = parsed_record["Ndarray"]
    npb = tf.io.decode_raw(npb,tf.int32)  # 注意dtype
    print(tf.reshape(npb,(2,4)))
    print(parsed_record["Image_raw"])
    # open("test2.png","wb").write(parsed_record["Image"].numpy()[0])

tf.Tensor([99], shape=(1,), dtype=int64)
tf.Tensor([12.364], shape=(1,), dtype=float32)
tf.Tensor([b'test'], shape=(1,), dtype=string)
tf.Tensor(
[[1 9 9 8]
 [5 2 5 8]], shape=(2, 4), dtype=int32)
tf.Tensor([b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x02\x00\x00\x00\xfdoH\xc3\x00\x00\x06\\IDATx\x9cmV\xdfo\x14\xe7\x15=\xdf\x8f\x99ov\xd73\xbb\xd8^c/;f\x89\x1d(6$)\xa9T\xe0\x01\x1a"RWU\xa3\xa4*\xf4\xc9\xea\x13o}\xe8?\xd0?\xa2\xcf\xc9C$\xde@\xaaT!\xa5 \xb7M\xd5J\xd0>PR\xc0\xa2\xc0\x82\rk\xaf\xb1w\xed\xf5\xcc\xeez\xe7\xe7w\xfb0\x8eqH\xaeF\x9a\x19it\xbes\xee=\xe7j\xd8\xe7\x7f\xf8\x10\xfb\xea\xd2\xfc\x02\x00)\xd9\xad\x1b\xbfm\xbfz\xc0\xb9A\x00\x83\xd6 \xd2\xd1\xf8\xd4\xcf[/\xfeV\x9e\xf8a\xb3q\xdb\xe09p\xc4b{\xeeg_\x94GN\xa4\x9a\xf6@$\x00\xdbq\x01(\xe5\x9c|\xffJ\x14\xfaB\x9a\x9b\xad\'\xa61$\xa4\xa5u\xc2\x18\x08Zp\x9e;P\xddj\xde\x16\xd2j\xbf\xfa\xafi\x14H\xa7BZc\xe5\xf3\x82\xc9T\'\x80\x08C\xbf\xeb7\xbe\xba\xf9;yi~\xa1\xd9\xb8\xdd\\\xb9sdz\xae0TIu\xd2\xf3\x9a\xcb\xf